In [1]:
devtools::install_github("NPSDC/beaveR", ref = "dev")
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))
clustFile <- "../brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold0/cluster_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
# tse <- buildTSE(treeTermFile = clustFile, coldata = coldata)

Skipping 5 packages not available: S4Vectors, tximeta, TreeSummarizedExperiment, SingleCellExperiment, SummarizedExperiment



── R CMD build ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
✔  checking for file ‘/cbcbhomes/npsingh/.tmp/Rtmp1Lf46w/remotesa8e746f5835/NPSDC-beaveR-55f74bc/DESCRIPTION’ (592ms)
─  preparing ‘beaveR’:
✔  checking DESCRIPTION meta-information
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘beaveR_0.99.0.tar.gz’
   


Warning message in i.p(...):
“installation of package ‘/cbcbhomes/npsingh/.tmp/Rtmp1Lf46w/filea8e7661d5d90/beaveR_0.99.0.tar.gz’ had non-zero exit status”


In [5]:
saveDir <- "environment/brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
# save(tse, file=file.path(saveDir, "tseCons.RData"))
load(file=file.path(saveDir, "tseCons.RData"))

In [7]:
load("environment/brain_sim_nodtu_lowfc/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))

tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])
tps <- which(abs(logFCNodes) > rootFC)

In [8]:
load(file=file.path(saveDir, "cutsCons.RData"))
load(file=file.path(saveDir, "negNodesCons.RData"))
load(file=file.path(saveDir, "detNodesCons.RData"))

In [9]:
y <- fishpond::scaleInfReps(tse[cuts[["lfcTD"]],])
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

In [26]:
length(cuts[["lfcTD"]])
length(cuts[["lfcTPMDesc"]])

[1] 79321

[1] 31545

In [23]:
# lfcTPM <- getTPMLFC(tse, "condition")
# descSize <- sapply(Descendants(tree, seq(nrow(tse))),length)
# metric <- abs(lfcTPM)*descSize
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cs <- objS[["cut"]]


In [25]:
length(cs)

[1] 79321

In [27]:
y <- fishpond::scaleInfReps(tse[cs,])
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")
dN <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
nN <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    if(!n%in%c("term", "Genes")) {
        print(n)
        stats <-  sapply(seq_along(dN[[n]]), function(i) computeMetOut(dN[[n]][[i]], logFCNodes, tree = tree, negNodes = nN[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        df <- rbind(df, stats)
        
    }
    
}
print(df)
comp

[1] "Txps"


ERROR: Error in `colnames<-`(`*tmp*`, value = colnames(df)[c(3:5)]): attempt to set 'colnames' on an object with less than two dimensions


In [42]:
devtools::install_github("NPSDC/beaveR", ref = "dev")
library(beaveR)
y2 <- computeSizeFactors(tse, type = "txp")
y2 <- scaleInfReps(y2)
y2 <- labelKeep(y2)
set.seed(1)
y2 <- swish(y2, x="condition")

Skipping install of 'beaveR' from a github remote, the SHA1 (55f74bc0) has not changed since last install.
  Use `force = TRUE` to force installation

Setting inf rep depth to mean depth



Progress:   1 on 100Progress:   2 on 100Progress:   3 on 100Progress:   4 on 100Progress:   5 on 100Progress:   6 on 100Progress:   7 on 100Progress:   8 on 100Progress:   9 on 100Progress:  10 on 100Progress:  11 on 100Progress:  12 on 100Progress:  13 on 100Progress:  14 on 100Progress:  15 on 100Progress:  16 on 100Progress:  17 on 100Progress:  18 on 100Progress:  19 on 100Progress:  20 on 100Progress:  21 on 100Progress:  22 on 100Progress:  23 on 100Progress:  24 on 100Progress:  25 on 100Progress:  26 on 100Progress:  27 on 100Progress:  28 on 100Progress:  29 on 100Progress:  30 on 100Progress:  31 on 100Progress:  32 on 100Progress:  33 on 100Progress:  34 on 100Progress:  35 on 100Progress:  36 on 100Progress:  37 on 100Progress:  38 on 100Progress:  39 on 100Progress:  40 on 100Progress:  41 on 100Progress:  42 on 100Progress:  43 on 100Progress:  44 on 100Progress:  45 on 100Progress:  46 on 100Progress:  47 on 100Progress:  48 on 100Progress:  49 on 100Progress:  50 on 100

In [44]:
suppressPackageStartupMessages(source("old_terminus_helper.R"))
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu_lowfc/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/"
# seBrainSim <- tximeta(coldata)
yTermThrNS <- tse[1:l,]
groupsClust <- parseClustFile(file.path(dir, "old", "1_1", "clusters.txt"), yTermThrNS)
mInds <- seq(nrow(yTermThrNS) + length(groupsClust))
yAggTermThrNS <- prepOSwish(yTermThrNS, mInds, groupsClust)

rownames(yAggTermThrNS)[l+1:length(groupsClust)] <- paste("Node", as.character(l+1:length(groupsClust)),sep="")
metadata(yAggTermThrNS)[["txpsAnn"]] <- rowData(yTermThrNS)
yAggTermThrNS <- yAggTermThrNS[-unlist(groupsClust),]
dim(yAggTermThrNS)

[1] 130016     12

In [48]:
yAggTermThrNS <- fishpond::scaleInfReps(yAggTermThrNS)
yAggTermThrNS <- labelKeep(yAggTermThrNS)
set.seed(1)
yAggTermThrNS <- swish(yAggTermThrNS, x="condition")

In [49]:
aggCountsNodes <- computeOAggNodesU(groupsClust, mInds, sim.counts.mat)
logFCNodesTerm <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)] <- paste("Node", names(logFCNodesTerm)[(l+1):length(logFCNodesTerm)], sep="")

In [50]:
desc <- unlist(Descendants(tree, cuts[["lfcTD"]]))

In [51]:
sum(mcols(y)[rownames(tse)[cuts[["lfcTD"]]],"qvalue"] < 0.1,na.rm=T)
sum(mcols(y2)[cuts[["lfcTD"]],"qvalue"] < 0.1,na.rm=T)

[1] 2444

[1] 2575

In [52]:
desc <- unlist(Descendants(tree, negNodes[["lfcTD"]][[3]]))
length(desc)
sum(desc %in% tps[tps < l])

[1] 122191

[1] 2797

In [12]:
summary(abs(mcols(y2)[["log2FC"]])[desc[desc %in% tps[tps < l]]])
t2 <- desc[which(mcols(y2)[desc,"qvalue"] < 0.1)] ## positive nodes of negative inner nodes
length(t2)
sum(Ancestors(tree, t2, "parent") == (l+1)) ## positive nodes that have root as parent
t2 <- t2[Ancestors(tree, t2, "parent") != (l+1)]
sum(t2 %in% tps) ## Txps that are false negative
t2 <- t2[t2 %in% tps]
t2 <- t2[!t2 %in% cuts[["lfcTPM"]]]
length(t2)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
0.00043 0.40797 0.65273 0.71039 0.95034 3.46999     176 

[1] 230

[1] 0

[1] 179

[1] 179

In [21]:
for(i in seq(10)) {
        print("new")
        print(mcols(y2)[c(t2[i], Ancestors(tree, t2[i])),])
        print(mcols(y2)[Ancestors(tree, t2[i])[Ancestors(tree, t2[i]) %in% cuts[["lfcTPM"]]],])
        
}


[1] "new"
DataFrame with 8 rows and 8 columns
                  meanInfRV log10mean      keep      stat    log2FC     pvalue
                  <numeric> <numeric> <logical> <numeric> <numeric>  <numeric>
ENST00000495891.1 4.5605685   2.44361      TRUE     15.29  1.119270 0.00773044
Node138608        4.0222251   2.66539      TRUE     13.37  1.124385 0.02023818
Node138607        1.0498666   2.77333      TRUE     13.83  0.977281 0.01632057
Node138606        0.3292864   2.79665      TRUE     13.05  0.966811 0.02352192
Node138605        0.2061407   2.80357      TRUE     13.00  0.957349 0.02412484
Node138604        0.1246748   2.80817      TRUE     13.21  0.948177 0.02180659
Node138603        0.0663178   2.81286      TRUE     13.22  0.943573 0.02170472
Node138290        0.0100000   7.49670      TRUE      2.93  0.016246 0.61764653
                     locfdr    qvalue
                  <numeric> <numeric>
ENST00000495891.1  0.369410 0.0902269
Node138608         0.660461 0.1871298
Node138607  

In [112]:
logFCNodes[c(t2[i],Siblings(tree, t2[i]))]

ENST00000495891.1 ENST00000542783.5 
         1.027061          1.028480

In [68]:
tpsTerm <- intersect(tps, c(detNodes[["term"]][[3]][detNodes[["term"]][[3]] <= l],
                       unlist(groupsClust[detNodes[["term"]][[3]][detNodes[["term"]][[3]] >= l] - l])))
tpsTPMTD <- intersect(tps, unlist(Descendants(tree, detNodes[["lfcTD"]][[3]])))
tpsTxp <- intersect(tps, detNodes[["Txps"]][[3]])
length(tpsTerm)
length(tpsTPMTD)
length(tpsTxp)
length(setdiff(tpsTerm, tpsTPMTD))
length(setdiff(tpsTxp, tpsTPMTD))
length(setdiff(tpsTPMTD, tpsTxp))
length(setdiff(tpsTPMTD, tpsTerm))
length(setdiff(tpsTPMTD, union(tpsTxp, tpsTerm)))

[1] 3935

[1] 5980

[1] 3575

[1] 278

[1] 300

[1] 2705

[1] 2323

[1] 2276

In [67]:
1

[1] 1

In [70]:
ancTPM <- cuts[["lfcTD"]][cuts[["lfcTD"]] %in% unique(unlist(Ancestors(tree, setdiff(tpsTPMTD, union(tpsTxp, tpsTerm)))))]
length(ancTPM)
length(unlist(Descendants(tree, ancTPM)))

[1] 828

[1] 9525

In [64]:
length(intersect(unlist(Descendants(tree, ancTPM)), tps))
length(setdiff(intersect(unlist(Descendants(tree, detInnTPMNodes)), tps), tpsTerm))

[1] 4285

[1] 2310

In [9]:
lfcTPM <- getScaledLFC(tse, "condition")
objS <- solveForOptimalCut(tse, metVec = abs(lfcTPM)*descSize, type = 'max')
cuts[["lfcTD"]] <- objS[["cut"]]
objS <- solveForOptimalCut(tse, metVec = abs(lfcTPM), type = 'max')
cuts[["lfcT"]] <- objS[["cut"]]

In [10]:
getLog2FC <- function(tse, condition, pc=5, array=FALSE) {
  infRepIdx <- grep("infRep", assayNames(tse))
  infReps <- assays(tse)[infRepIdx]
  infRepsArray <- abind::abind(as.list(infReps), along=3)
  dims <- dim(infRepsArray)
    
  cond1 <- condition == levels(condition)[1]
  cond2 <- condition == levels(condition)[2]

  diffs <- matrix(nrow=dims[1],ncol=dims[3])
  for (k in seq_len(dims[3])) {
    diffs[,k] <- log2(rowMeans(infRepsArray[,cond2,k]) + pc) -
                 log2(rowMeans(infRepsArray[,cond1,k]) + pc)
  }
  if (array) {
    return(diffs)
  }
  # median over inferential replicates
  rowMedians(diffs)
}

In [52]:
sfTxp <-  metadata(computeSizeFactors(tse, type="txp"))[["sf"]]
sfGene <-  metadata(computeSizeFactors(tse, type="gene"))[["sf"]]
head(sfTxp)
head(sfGene)
# metric <- abs(lfc)
# objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
# cuts[["LFC"]] <- objS[["cut"]]

# metric <- abs(lfc)*descSize
# objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
# cuts[["LFCD"]] <- objS[["cut"]]

loading existing TxDb created: 2022-08-19 22:12:59

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2022-08-26 02:08:58

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



1.061562,1.044234,1.075860,1.068837,1.072721,1.083489,1.069820,1.082284,1.076877,1.067842,1.066236,1.019739
1.058253,1.045463,1.080662,1.066894,1.075540,1.081470,1.071871,1.081743,1.078637,1.068676,1.064942,1.022908
1.059917,1.043573,1.078233,1.069372,1.074577,1.080657,1.070616,1.083024,1.077796,1.070355,1.066477,1.019795
1.060829,1.043854,1.075131,1.068136,1.072915,1.081361,1.071144,1.083654,1.079466,1.068916,1.065524,1.023065
1.062674,1.046074,1.081494,1.068471,1.073142,1.081804,1.071203,1.082600,1.079018,1.068460,1.063230,1.020367
1.061258,1.045369,1.080298,1.067462,1.071060,1.085672,1.067514,1.085747,1.076454,1.068918,1.063508,1.020668


1.002113,0.9883031,1.020161,1.007615,1.018721,1.032465,1.011108,1.024619,1.019133,1.009610,1.015654,0.9707576
1.004378,0.9869173,1.019453,1.007202,1.021940,1.030819,1.009343,1.023453,1.019948,1.009705,1.012208,0.9704418
1.003430,0.9869121,1.019695,1.006203,1.021818,1.030183,1.010917,1.024761,1.021206,1.012103,1.012285,0.9703761
1.004075,0.9885824,1.018313,1.005689,1.018024,1.029205,1.006626,1.023705,1.016882,1.010631,1.015338,0.9716320
1.002327,0.9863272,1.019272,1.004953,1.019325,1.029188,1.009281,1.022419,1.020187,1.008550,1.015466,0.9710223
1.004711,0.9862580,1.019774,1.007414,1.019463,1.030103,1.008235,1.023228,1.020001,1.010584,1.011943,0.9713567


In [60]:
# txi <- tximport(files = coldata$files, type="salmon", txOut = T)
dds <- DESeqDataSetFromTximport(txi)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'nrow': argument "colData" is missing, with no default


In [27]:
# detNodes <- list()
# negNodes <- list()
# for(n in names(cuts)) {
#     y <- tse[sort(cuts[[n]]),]
#     y <- fishpond::scaleInfReps(y)
#     y <- labelKeep(y)
#     set.seed(1)
#     y <- swish(y, x="condition")
#     detNodes[[n]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
#         nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
#         match(nodes, names(logFCNodes))
#     })
#     negNodes[[n]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
#             nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
#             match(nodes, names(logFCNodes))
#         })
# }
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(cuts)) {
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        print(stats)                 
        df <- rbind(df, stats)
}

[1] "tp 0.993719469644103"
[1] "tp 0.962639495390587"
[1] "tp 0.919328419328419"
    Method  Metric FDR_0.01   FDR_0.05  FDR_0.10  
fdr "lfcTD" "FDR"  0.00628053 0.0373605 0.08067158
tpr "lfcTD" "TPR"  0.3140022  0.4374862 0.4950386 
[1] "tp 0.992303463441451"
[1] "tp 0.962640901771337"
[1] "tp 0.922936259143156"
    Method Metric FDR_0.01    FDR_0.05  FDR_0.10  
fdr "lfcT" "FDR"  0.007696537 0.0373591 0.07706374
tpr "lfcT" "TPR"  0.2056511   0.3405492 0.4025293 
[1] "tp 0.991052993805919"
[1] "tp 0.955938697318008"
[1] "tp 0.909127462806594"
    Method Metric FDR_0.01    FDR_0.05  FDR_0.10  
fdr "LFCD" "FDR"  0.008947006 0.0440613 0.09087254
tpr "LFCD" "TPR"  0.3197158   0.4431616 0.5019982 


In [29]:
negNodesDesc <- list()
for(n in names(negNodes)) {
    if(n == "Swish")
        negNodesDesc[[n]] <- negNodes[[n]]
    if(n != "term")
        negNodesDesc[[n]] <- lapply(negNodes[[n]], function(x) unlist(Descendants(tree, x)))
}
dfD <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes)) {
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodesDesc[[n]][[i]], lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        print(stats)                 
        dfD <- rbind(dfD, stats)
}
df

[1] "tp 0.993719469644103"
[1] "tp 0.962639495390587"
[1] "tp 0.919328419328419"
    Method  Metric FDR_0.01   FDR_0.05  FDR_0.10  
fdr "lfcTD" "FDR"  0.00628053 0.0373605 0.08067158
tpr "lfcTD" "TPR"  0.2539683  0.3829377 0.4452598 
[1] "tp 0.992303463441451"
[1] "tp 0.962640901771337"
[1] "tp 0.922936259143156"
    Method Metric FDR_0.01    FDR_0.05  FDR_0.10  
fdr "lfcT" "FDR"  0.007696537 0.0373591 0.07706374
tpr "lfcT" "TPR"  0.2056511   0.3405492 0.4025293 
[1] "tp 0.991052993805919"
[1] "tp 0.955938697318008"
[1] "tp 0.909127462806594"
    Method Metric FDR_0.01    FDR_0.05  FDR_0.10  
fdr "LFCD" "FDR"  0.008947006 0.0440613 0.09087254
tpr "LFCD" "TPR"  0.2592259   0.3875728 0.4521096 


,Method,Metric,FDR_0.01,FDR_0.05,FDR_0.10
,<named list>,<named list>,<named list>,<named list>,<named list>
fdr,lfcTD,FDR,0.00628053,0.0373605,0.08067158
tpr,lfcTD,TPR,0.3140022,0.4374862,0.4950386
fdr1,lfcT,FDR,0.007696537,0.0373591,0.07706374
tpr1,lfcT,TPR,0.2056511,0.3405492,0.4025293
fdr2,LFCD,FDR,0.008947006,0.0440613,0.09087254
tpr2,LFCD,TPR,0.3197158,0.4431616,0.5019982


In [28]:
suppressPackageStartupMessages(library(tidyr))
dfA <- df %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
colnames(dfA)[c(2:7)] <- as.vector(outer(c(0.01,0.05,0.1), c("FDR","TPR"), function(x,y) paste(x,y,sep="_")))
for(i in seq(ncol(dfA)-1))
    dfA[,i+1] <- round(as.numeric(unlist(dfA[,i+1])),3)
dfA <- dfA[order(unlist(dfA$Method)),]
dfA

Method,0.01_FDR,0.05_FDR,0.1_FDR,0.01_TPR,0.05_TPR,0.1_TPR
<named list>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LFCD,0.009,0.044,0.091,0.320,0.443,0.502
lfcT,0.008,0.037,0.077,0.206,0.341,0.403
lfcTD,0.006,0.037,0.081,0.314,0.437,0.495


In [23]:
suppressPackageStartupMessages(library(tidyr))
dfA <- dfD %>% pivot_wider(names_from = Metric, values_from = c(3:5))
dfA <- dfA[,c(1,seq(2,7,2),seq(3,7,2))]
colnames(dfA)[c(2:7)] <- as.vector(outer(c(0.01,0.05,0.1), c("FDR","TPR"), function(x,y) paste(x,y,sep="_")))
for(i in seq(ncol(dfA)-1))
    dfA[,i+1] <- round(as.numeric(unlist(dfA[,i+1])),3)
dfA <- dfA[order(unlist(dfA$Method)),]
dfA

Method,0.01_FDR,0.05_FDR,0.1_FDR,0.01_TPR,0.05_TPR,0.1_TPR
<named list>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
LFCD,0.009,0.044,0.091,0.259,0.388,0.452
lfcT,0.008,0.037,0.077,0.206,0.341,0.403
lfcTD,0.006,0.037,0.081,0.254,0.383,0.445


In [30]:
cutsY <- cuts
detNodesY <- detNodes
negNodesY <- negNodes

In [36]:
load(file=file.path(saveDir, "cutsCons.RData"))
load(file=file.path(saveDir, "negNodesCons.RData"))
load(file=file.path(saveDir, "detNodesCons.RData"))

In [38]:
cuts <- append(cuts, cutsY)
detNodes <- append(detNodes, detNodesY)
negNodes <- append(negNodes, negNodesY)

In [40]:
save(cuts, file = file.path(saveDir, "cutsCons.RData"))
save(negNodes, file = file.path(saveDir, "negNodesCons.RData"))
save(detNodes, file = file.path(saveDir, "detNodesCons.RData"))